In [1]:
import pandas as pd 

In [2]:
rawTranslated = pd.read_csv("/shared/3/projects/benlitterer/localNews/NetworkMVP/allTrainDataTranslated.tsv", sep="\t")

In [5]:
rawTranslated.head(3)

,Unnamed: 0,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,...,id1,id2,text1,title1,text2,title2,title1En,title2En,text1En,text2En
0,0.0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-...,https://www.washingtonpost.com/world/the_ameri...,https://web.archive.org/web/www.washingtonpost...,https://web.archive.org/web/www.washingtonpost...,4.0,4.0,...,1.484084e+09,1.484110e+09,"MARTINSBURG, W.Va. — A suspected drunken drive...",Virginia man arrested in fatal DUI crash in We...,"PORT-AU-PRINCE, Haiti — Haitian President Jove...",Haiti’s leader marks independence day amid sec...,Virginia man arrested in fatal DUI crash in We...,Haiti’s leader marks independence day amid sec...,"MARTINSBURG, W.Va. — A suspected drunken drive...","PORT-AU-PRINCE, Haiti — Haitian President Jove..."
1,1.0,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three...,https://www.thestar.com/news/world/europe/2020...,https://web.archive.org/web/www.stlucianewsonl...,https://web.archive.org/web/www.thestar.com/ne...,4.0,4.0,...,1.484396e+09,1.483925e+09,Share This On: Pin 11 Shares (NEWS ROOM GUYA...,Guyana: Three injured after car crashes into u...,BERLIN - A fire at a zoo in western Germany in...,Fire kills more than 30 animals at zoo in west...,Guyana: Three injured after car crashes into u...,Fire kills more than 30 animals at zoo in west...,Share This On: Pin 11 Shares (NEWS ROOM GUYA...,BERLIN - A fire at a zoo in western Germany in...
2,2.0,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-...,https://www.timesofisrael.com/trump-says-he-do...,https://web.archive.org/web/www.teaparty.org/t...,https://web.archive.org/web/www.timesofisrael....,1.0,2.0,...,1.484698e+09,1.483759e+09,(Breitbart) – President Donald Trump welcomed ...,Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...,"PALM BEACH, United States — US President Donal...","Trump says he does not expect war with Iran, ‘...",Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...,"Trump says he does not expect war with Iran, ‘...",(Breitbart) – President Donald Trump welcomed ...,"PALM BEACH, United States — US President Donal..."


Get columns with original english or translated english when appropriate

In [40]:
#created nestedList for pandas dataframe 
dfList = []
for i, row in rawTranslated.iterrows(): 
    
    if row["url1_lang"] == "en": 
        text1 = row["text1"]
        title1 = row["title1"]
    else: 
        text1 = row["text1En"]
        title1 = row["title1En"]
        
    if row["url2_lang"] == "en": 
        text2 = row["text2"]
        title2 = row["title2"]
    else: 
        text2 = row["text2En"]
        title2 = row["title2En"]
    
    newRow = list(row[1:21]) + [text1, title1, text2, title2]
    dfList.append(newRow)
    
names = list(row[1:17].index) + ["ogText1", "ogTitle1", "ogText2", "ogTitle2", "text1", "title1", "text2", "title2"] 
mergedData = pd.DataFrame(dfList, columns=names)

clean things up a bit 

In [51]:
print(mergedData.shape)
cleanedMerged = mergedData[(mergedData["text1"] != "#VALUE!") & (mergedData["text2"] != "#VALUE!")]
cleanedMerged = cleanedMerged.dropna(subset=["text1", "text2"])

#turn everything that is junk to empty string. Hopefully in this case we just get padding 
#that goes through the network and doesn't change everything 
cleanedMerged[["title1", "title2"]] = cleanedMerged[["title1","title2"]].fillna("").replace("#VALUE!", "")

print(cleanedMerged.shape)

(4922, 24)
(4876, 24)


checking language distribution mostly out of curiousity 

In [56]:
cleanedMerged["url1_lang"].value_counts()

en    1767
de    1403
es     561
tr     455
pl     349
ar     269
fr      72
Name: url1_lang, dtype: int64

In [57]:
cleanedMerged["url2_lang"].value_counts()

en    2325
de     845
es     561
tr     455
pl     349
ar     269
fr      72
Name: url2_lang, dtype: int64

In [59]:
cleanedMerged.to_csv("/shared/3/projects/benlitterer/localNews/NetworkMVP/translatedCleaned.tsv", sep="\t")